In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
# File containing the schema of each launch_file
launch_schema = pd.read_csv('D:/Bureau/Stage/BCG\PwC/space-launches/schemas/launch_schema.csv')

# DataFrame that will contain all the data and that will be transformed into a .csv file
launches_csv = pd.DataFrame(columns = launch_schema.column)

In [123]:
# ## Rename all the files with a .txt extension
# ## Careful, do it only once

# folder = 'D:\Bureau\Stage\BCG\PwC\space-launches\launches'
# files = os.listdir(folder)

# for _, file in enumerate(files):
#     os.rename(os.path.join(folder, file), os.path.join(folder, file + '.txt'))

In [139]:
## A function that extracts all the lines in a .txt file in the form of a list
def extract_all_lines(folder, file):
    file_open = open(os.path.join(folder, file), 'r')
    lines = file_open.readlines() 
    lines_read = []
    for line in lines:
        lines_read.append(line.strip())
        
    return(lines_read)

In [125]:
## For each file, read its lines and append them to launches_csv in the right columns
def complete_launches_csv(lines_read, launches_csv):
    # Ignore the following lines 
    # The 'Launch_tag' line is just the headers
    for line in lines_read:
        if (line == '<PRE>') or (line[0:10] == 'Launch_Tag') or (line == '</PRE>'):
            continue
            
        # Use launch_schema.csv 
        # We are looking for the position of the data in the line, which is a string of characters 
        else:
            line_to_append = {}
            for category in launch_schema.column:
                start  = int(launch_schema[launch_schema['column']==category]['start']-1)   # Start of the data in the line
                length = int(launch_schema[launch_schema['column']==category]['length'])
                end    = start + length                                                     # End of the data in the line
                line_to_append[category] = line[start:end]                                  
                
            launches_csv = launches_csv.append(pd.Series(line_to_append), ignore_index=True)
    return(launches_csv)

In [140]:
## For each file, extract its lines
## Then complete launches_csv
# Don't run it again if the extraction was already done

# folder = 'D:\Bureau\Stage\BCG\PwC\space-launches\launches'
# files = os.listdir(folder)

# # tqdm gives a progress bar 

# for file in tqdm(files):
#     lines_read = extract_all_lines(folder, file)
#     launches_csv = complete_launches_csv(lines_read, launches_csv)

100%|██████████████████████████████████████████████████████████████████████████████| 331/331 [2:55:28<00:00, 31.81s/it]


In [181]:
## Save launches_csv to a .csv file

# launches_csv.to_csv('D:/Bureau/Stage/BCG/PwC/space-launches/results/launches_csv.csv')

In [3]:
## Read launches_csv.csv

launches_csv = pd.read_csv('D:/Bureau/Stage/BCG/PwC/space-launches/results/launches_csv.csv')

In [4]:
## Drop useless columns

launches_csv.drop(columns=['tag', 'date', 'variant', 'flight_id',
                           'flight', 'mission', 'flight_code', 'platform', 'launch_site',
                           'launch_pad', 'apogee_km', 'apoflag', 'range', 'range_flag',
                           'destination', 'agency', 'launch_code', 'launch_group', 'category',
                           'citation1', 'citation2', 'notes'], inplace=True)

In [5]:
## Create a column 'year' using the julian_date

import datetime

# Epoch of reference
epoch = pd.to_datetime(0, unit='s').to_julian_date()

# A series containing the column 'julian_date' converted into float
julian_date = launches_csv['julian_date'].map(lambda x: float(x))

# A series containing the 'julian_date' transformed into readable date (yyyy-mm-dd)
date = pd.to_datetime(julian_date - epoch, unit='D')

# Create a column with the year of each launch
launches_csv['year'] = date.map(lambda x: x.year)

In [6]:
## Function that writes the content of a dataframe to an Excel file

import xlsxwriter
import openpyxl

def write_excel(year, df, file):
    
    # Verify if the excel file exists. If not, creates it
    if not os.path.exists(file): 
        workbook = openpyxl.Workbook(file)
        workbook.save(file)
        
    excel_book = openpyxl.load_workbook(file)
      
    # Open the Excel file and writes in it
    with pd.ExcelWriter(file, engine='openpyxl') as writer:
        writer.book = excel_book
        writer.sheets = {
            worksheet.title: worksheet
            for worksheet in excel_book.worksheets
        }
        df.to_excel(writer, str(year), index=False)
        writer.save()

In [7]:
## Change the name of rockets 

def change_rocket_name(df):    
    dict_change_rocket_name = {
    'lv_type': {
        'Chang Zheng 3B           ': 'CZ - 3',
        'Chang Zheng 4B           ': 'CZ - 4',
        'Chang Zheng 11           ': 'CZ - 11',
        'Chang Zheng 4C           ': 'CZ - 4',
        'Proton-M/Briz-M          ': 'Proton-M',
        'Chang Zheng 3B/YZ-1      ': 'CZ - 3',
        'Ariane 5ECA              ': 'Ariane 5',
        'Soyuz-ST-B               ': 'Soyuz-ST', 
        'Soyuz-U-PVB              ': 'Soyuz-2',
        'Proton-M/DM-3            ': 'Proton-M',
        'Soyuz-2-1A               ': 'Soyuz-2',
        'Soyuz-2-1B               ': 'Soyuz-2',
        'Soyuz-2-1V               ': 'Soyuz-2',
        'Chang Zheng 2D           ': 'CZ - 2',
        'Delta 4M+(5,2)           ': 'Delta-4 Medium',
        'Chang Zheng 5            ': 'CZ - 5',
        'Chang Zheng 2C           ': 'CZ - 2',
        'Delta 4H                 ': 'Delta-4 Heavy',
        'Soyuz-ST-A               ': 'Soyuz-ST',
        'Atlas V 551              ': 'Atlas V',
        'Delta 4M+(4,2)           ': 'Delta-4 Medium',
        'Atlas V N22              ': 'Atlas V',
        'Chang Zheng 3C           ': 'CZ - 3',
        'Chang Zheng 6            ': 'CZ - 6',
        'Ariane 5ECA+             ': 'Ariane 5',
        'Ariane 5ES               ': 'Ariane 5',
        'Atlas V 541              ': 'Atlas V',
        'Terrier Imp Malemute     ': 'Malemute',
        'Improved Orion           ': 'Orion',
        'Improved Malemute        ': 'Malemute',
        'Terrier Malemute         ': 'Malemute',
        'Terrier Improved Orion   ': 'Orion',
        'Soyuz-FG                 ': 'Soyuz-2',
        'PSLV-XL                  ': 'PLSV',
        'PSLV                     ': 'PLSV',
        'Delta 4H/Star 48BV       ': 'Delta-4 Heavy',
        'Atlas V 411              ': 'Atlas V',
        'Atlas V 431              ': 'Atlas V',
        'VS-31/Imp Malemute       ': 'Malemute',
        'Chang Zheng 2C/YZ-1S     ': 'CZ - 2',
        'Chang Zheng 2D/YZ-3      ': 'CZ - 2',
        'Chang Zheng 7/YZ-1A      ': 'CZ - 7',
        'PSLV-DL                  ': 'PLSV',
        'PSLV-QL                  ': 'PLSV',
        'Atlas V 401              ': 'Atlas V',
        'Antares 230+             ': 'Antares',
        'Antares 230              ': 'Antares',
        'Antares 130              ': 'Antares',
        'Antares 120              ': 'Antares',
        'Ariane 5ES/ATV           ': 'Ariane 5',
        'Black Brant IX           ': 'Black Brant',
        'Black Brant XIA          ': 'Black Brant',
        'Black Brant XIIA         ': 'Black Brant',
        'Chang Zheng 3A           ': 'CZ - 3',
        'GSLV Mk II               ': 'GLSV',
        'GSLV Mk III              ': 'GLSV',
        'H-IIA 202                ': 'H-II',
        'H-IIB                    ': 'H-II',
        'OS-X1                    ': 'OS-X',
        'OS-X2                    ': 'OS-X',
        'Chang Zheng 2F           ': 'CZ - 2',
        'IM/VS-30                 ': 'VS-30',
        'VS-30/Orion              ': 'VS-30',
        'Chang Zheng 3C/YZ-1      ': 'CZ - 3',
        'Black Brant X            ': 'Black Brant',
        'OS-X0                    ': 'OS-X',
        'Atlas V 421              ': 'Atlas V',
        'H-IIA 204                ': 'H-II',
        'Terrier Oriole           ': 'Orion',
        'Terrier Terrier Oriole   ': 'Orion',
        'Delta 4M+(5,4)           ': 'Delta-4 Medium',
        'VS-30                    ': 'VS-30',
        'Chang Zheng 7            ': 'CZ - 7',
        'Chang Zheng 5/YZ-2       ': 'CZ - 5',
        'Black Brant XI           ': 'Black Brant',
        'Atlas V 501              ': 'Atlas V',
        'Terrier Mk 12 Imp Orion  ': 'Orion',
        'Terrier Orion            ': 'Orion',
        'VS-31/Orion              ': 'Orion',
        'Angara A5                ': 'Angara',
        'Angara-1.2PP             ': 'Angara',
        'Falcon 9R                ': 'Falcon 9',
        'Kuaizhou                 ': 'Kuaizhou',
        'Kuaizhou-1A              ': 'Kuaizhou',
        'Terrier Lynx             ': 'Malemute'
        }
    }
    
    df.replace(dict_change_rocket_name, inplace=True)
    return(df)
    
launches_csv = change_rocket_name(launches_csv)

In [8]:
## Drop the rows that were already identified as not rockets from 2016 - 2019
    
list_not_rocket = [
 'Agni 1                   ',
 'Agni 2                   ',
 'Agni 5                   ',
 'Arrow 3                  ',
 'Astra                    ',
 'B-611                    ',
 'Badr-1                   ',
 'Boosted Zombie           ',
 'Bulava                   ',
 'Burkan-2H                ',
 'Dhanush                  ',
 'eMRBM                    ',
 'Fateh-110                ',
 'FTB                      ',
 'FTI                      ',
 'Ghauri                   ',
 'Grad                     ',
 'IRBM-T1                  ',
 'JL-3                     ',
 'K-15                     ',
 'Khorramshahr             ',
 'Kinzhal                  ',
 'Lynx                     ',
 'Minuteman 3              ',
 'New Shepard              ',
 'Nucleus                  ',
 "Nudol'                   ",
 'PDV                      ',
 'PDV Target               ',
 'Prithvi 2                ',
 'PRS-1M                   ',
 'Qahir-1                  ',
 'Qahir-M2                 ',
 'Qiam-1                   ',
 'RH-300 Mk II             ',
 'R-UNK                    ',
 'Sineva                   ',
 'SM-3-IB                  ',
 'SM-3-IIA                 ',
 'Spaceship Two            ',
 'SS-520                   ',
 'Tochka                   ',
 "Topol'                   ",
 "Topol'-E                 ",
 "Topol'-M                 ",
 'Trident D-5              ',
 'UR-100NU 15A35P          ',
 'Yars-M                   ',
 'Zelzal-1                 ',
 'Zelzal-2                 ',
 'Zulfiqar                 ',
 'Agni 3                   ',
 'Bukguekseong-3           ',
 'Burkan-3                 ',
 'DF-41                    ',
 'GBI                      ',
 'ICBM-T2                  ',
 'KN-23                    ',
 'KN-24                    ',
 'KN-25                    ',
 'MRT                      ',
 'PDV Mk II                ',
 'Safir                    ',
 'Shahab 3                 ',
 'Shaheen 2                ',
 'THAAD                    ',
 'Yars                     ',
 'Hwasong-12               ',
 'Hwasong 9                ',
 'Hyunmoo-2B               ',
 'Hyunmoo-2C               ',
 'Zelzal-3                 ',
 'Al Samoud                ',
 'Hwasong 5                ',
 'Hyunmoo-2A               ',
 'Sabre Zombie             ',
 'Bukguekseong-2           ',
 'Iskander-M               ',
 'Prithvi                  ',
 'Hwasong-14               ',
 'SM-6                     ',
 'AAD                      ',
 'ATACMS                   ',
 'Janus                    ', 
 'Arrow 2                  ',
 'Dong Feng 5C             ',
 'Zenit-3SLBF              ',
 'Agni 4                   ',
 'Minotaur IV              ',
 'Hwasong-15               ',
 'STARS                    ',
 'Ababeel                  ',
 'Black Sparrow            ',
 'DF-4                     ',
 'Burkan-1H                ',
 'MRBM-T3                  ',
 'Hwasong-10               ',
 'Maxus                    ',
 'Hwasong-7                ',
 'Hwasong 6                ',
 'Bukguekseong             ',
 'MRBM Target              ',
 'Ghadr 1                  ',
 'Pathfinder Zombie        ',
 'K-4                      ',
 'Shahab 1                 ',
 'Shahab 2                 ',
 'Qiam 1                   ',
 'Kwangmyongsong           ',
 'MSBS M51                 ',
 'DF-15                    ',
 'ATV                      ',
 'HS9                      ',
 'Perimeter-RTs            ',
 'Silver Sparrow           ',
 'DF-21                    ',
 'PRS-1                    ',
 'S-310                    ',
 'Tianying-3F              ',
 'Dnepr                    ',
 'Emad                     ',
 'JL-2                     ',
 'Juno                     ',
 'KN-02                    ',
 'Lance                    ',
 'LDSD                     ', 
 'Rubezh                   ',
 'Shaheen 1A               ',
 'Shaheen 3                ',
 'SM-2-IV                  ',
 'SM-3                     ',
 'SRALT                    ',
 'Yars 15Zh65M             ',
 'Blue Sparrow             ', 
 'CSXT                     ', 
 'DF-31                    ',
 'DF-21                    ',
 'Ghaznavi                 ',
 'K-BATS                   ',
 'LV-2                     ',
 'MN-300                   ',
 'RH-200                   ',
 'SM-2MR                   ',
 'Sounding Rocket          ',
 'Taurion                  ',
 'Zenit-3SL                '
 ]

row_not_rocket = launches_csv['lv_type'].isin(list_not_rocket)
row_not_rocket = launches_csv[row_not_rocket].index
launches_csv.drop(row_not_rocket, inplace=True)
launches_csv.reset_index(drop=True, inplace=True)

In [9]:
# file = 'D:/Bureau/Stage/BCG/PwC/space-launches/results/launches_csv_processed.xlsx'
# write_excel('all', launches_csv, file)

In [10]:
## All the launches in the chosen year
# Contains missiles launches that we have to filter one by one 

# File where we will save the launches
file = "D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx"

def launches_year_function(launches_csv, year):
    launches_year = launches_csv.loc[launches_csv['year']==year, 'lv_type'].value_counts()
    launches_year = launches_year.reset_index()
    launches_year.columns=(['Rocket', 'Number of launches'])
    
    # Add empty columns
    launches_year['Is_a_rocket'] = np.nan
    launches_year['Type'] = np.nan
    launches_year['Family'] = np.nan
    launches_year['LEO_payload'] = np.nan
    launches_year['Country'] = np.nan
    launches_year['Continent'] = np.nan

    #     Write all the launches in an excel file 
    write_excel(year, launches_year, file)
    return(launches_year)

In [11]:
launches_2019 = launches_year_function(launches_csv, 2019)
launches_2018 = launches_year_function(launches_csv, 2018)
launches_2017 = launches_year_function(launches_csv, 2017)
launches_2016 = launches_year_function(launches_csv, 2016)
launches_2015 = launches_year_function(launches_csv, 2015)
launches_2014 = launches_year_function(launches_csv, 2014)

**At this stage, filter by hand the missiles and the rockets by writing in the column Is_a_rocket**
<br>
**If it is a rocket: 1, if not: 0**
<br> <br>
**Then run the following lines**

In [53]:
## Read the excel file launch_vehicles (customized database)

launch_vehicles_2019 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2019')
launch_vehicles_2018 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2018')
launch_vehicles_2017 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2017')
launch_vehicles_2016 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2016')
launch_vehicles_2015 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2015')
launch_vehicles_2014 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2014')

## I've added 2020 myself
launch_vehicles_2020 = pd.read_excel('D:\Bureau\Stage\BCG\PwC\launch_vehicles.xlsx', sheet_name='2020')

In [54]:
## Drop the rows of launch vehicles that are missiles and not rockets

def drop_not_a_rocket(launch_vehicles_year, year):
    # Retrieve the column Is_a_rocket 
    is_a_rocket_column = launch_vehicles_year['Is_a_rocket']
    
    # Index of the rows that are not rockets
    index_is_not_a_rocket = launch_vehicles_year.loc[is_a_rocket_column==0].index
    
    # Drop these rows
    launch_vehicles_year.drop(index_is_not_a_rocket, inplace=True)
    
    # Drop the row Is_a_rocket
    launch_vehicles_year.drop(columns=['Is_a_rocket'], inplace=True)
    
    # Add a column with the year
    launch_vehicles_year['year'] = year
    
    # Reindex the rows
    launch_vehicles_year.reset_index(drop=True, inplace=True)
    return(launch_vehicles_year)

In [55]:
## Apply the drop_not_a_rocket function

launch_vehicles_processed_2020 = drop_not_a_rocket(launch_vehicles_2020, 2020)
launch_vehicles_processed_2019 = drop_not_a_rocket(launch_vehicles_2019, 2019)
launch_vehicles_processed_2018 = drop_not_a_rocket(launch_vehicles_2018, 2018)
launch_vehicles_processed_2017 = drop_not_a_rocket(launch_vehicles_2017, 2017)
launch_vehicles_processed_2016 = drop_not_a_rocket(launch_vehicles_2016, 2016)
launch_vehicles_processed_2015 = drop_not_a_rocket(launch_vehicles_2015, 2015)
launch_vehicles_processed_2014 = drop_not_a_rocket(launch_vehicles_2014, 2014)

In [56]:
## Export to an Excel file

file = 'D:\Bureau\Stage\BCG\PwC\launch_vehicles_processed.xlsx'

# A single table with all the rows
launch_vehicles_processed = launch_vehicles_processed_2020.append(launch_vehicles_processed_2019, ignore_index=True)
launch_vehicles_processed = launch_vehicles_processed.append(launch_vehicles_processed_2018, ignore_index=True)
launch_vehicles_processed = launch_vehicles_processed.append(launch_vehicles_processed_2017, ignore_index=True)
launch_vehicles_processed = launch_vehicles_processed.append(launch_vehicles_processed_2016, ignore_index=True)
launch_vehicles_processed = launch_vehicles_processed.append(launch_vehicles_processed_2015, ignore_index=True)
launch_vehicles_processed = launch_vehicles_processed.append(launch_vehicles_processed_2014, ignore_index=True)

write_excel(2020_2014, launch_vehicles_processed, file)

# Add other sheets
write_excel(2020, launch_vehicles_processed_2019, file)
write_excel(2019, launch_vehicles_processed_2019, file)
write_excel(2018, launch_vehicles_processed_2018, file)
write_excel(2017, launch_vehicles_processed_2017, file)
write_excel(2016, launch_vehicles_processed_2016, file)
write_excel(2016, launch_vehicles_processed_2015, file)
write_excel(2016, launch_vehicles_processed_2014, file)

In [57]:
## For the rockets having a number of launches <= threshold_nb_launches, rename as 'Other'

# Calculate the total number of launches over the period
launches_per_rocket = launch_vehicles_processed.groupby(['Rocket'])['Number of launches'].sum()

# Give the indices of the rockets to rename as 'Other'
threshold_nb_launches = 3
rocket_to_rename = launches_per_rocket[launches_per_rocket<=threshold_nb_launches].index

def rename_others(launch_vehicles_processed, index_to_rename):
    index_to_rename = launch_vehicles_processed['Rocket'].isin(index_to_rename)
    launch_vehicles_processed.loc[index_to_rename, 'Rocket'] = 'Other'
    launch_vehicles_processed_others = launch_vehicles_processed
    return(launch_vehicles_processed_others)

In [58]:
## Rename as the rockets as 'Other'

launch_vehicles_processed_others_2020 = rename_others(launch_vehicles_processed_2020, rocket_to_rename)
launch_vehicles_processed_others_2019 = rename_others(launch_vehicles_processed_2019, rocket_to_rename)
launch_vehicles_processed_others_2018 = rename_others(launch_vehicles_processed_2018, rocket_to_rename)
launch_vehicles_processed_others_2017 = rename_others(launch_vehicles_processed_2017, rocket_to_rename)
launch_vehicles_processed_others_2016 = rename_others(launch_vehicles_processed_2016, rocket_to_rename)
launch_vehicles_processed_others_2015 = rename_others(launch_vehicles_processed_2015, rocket_to_rename)
launch_vehicles_processed_others_2014 = rename_others(launch_vehicles_processed_2014, rocket_to_rename)

In [59]:
## Write in the file launch_vehicles_processed_others.xlsx

file = 'D:\Bureau\Stage\BCG\PwC\launch_vehicles_processed_others.xlsx'

# A single table with all the rows
launch_vehicles_processed_others = launch_vehicles_processed_others_2020.append(launch_vehicles_processed_others_2019, ignore_index=True)
launch_vehicles_processed_others = launch_vehicles_processed_others.append(launch_vehicles_processed_others_2018, ignore_index=True)
launch_vehicles_processed_others = launch_vehicles_processed_others.append(launch_vehicles_processed_others_2017, ignore_index=True)
launch_vehicles_processed_others = launch_vehicles_processed_others.append(launch_vehicles_processed_others_2016, ignore_index=True)
launch_vehicles_processed_others = launch_vehicles_processed_others.append(launch_vehicles_processed_others_2015, ignore_index=True)
launch_vehicles_processed_others = launch_vehicles_processed_others.append(launch_vehicles_processed_others_2014, ignore_index=True)


write_excel(2020_2014, launch_vehicles_processed_others, file)
write_excel(2020, launch_vehicles_processed_others_2020, file)
write_excel(2019, launch_vehicles_processed_others_2019, file)
write_excel(2018, launch_vehicles_processed_others_2018, file)
write_excel(2017, launch_vehicles_processed_others_2017, file)
write_excel(2016, launch_vehicles_processed_others_2016, file)
write_excel(2015, launch_vehicles_processed_others_2015, file)
write_excel(2014, launch_vehicles_processed_others_2014, file)

In [128]:
with pd.option_context('max_rows', 100):
    display(launches_2018.sort_values(by='Number of launches', ascending=False))

,Rocket,Number of launches
0,Badr-1,99
1,Zelzal-1,76
2,Zelzal-2,27
3,Falcon 9,20
4,Qahir-M2,19
...,...,...
80,Agni 2,1
81,GSLV Mk III,1
82,Fateh-110,1
83,H-IIB,1


In [ ]:
# Zelzal-1 is a missile


In [13]:
launches_csv.groupby[['lv_type'].value_counts()

Rocketsonde                  21362
M-100                         5880
M-100B                        1838
Loki Dart                     1593
Arcas                         1461
                             ...  
E-LRALT                          1
Dolphin                          1
Seagull                          1
KSLV-II TLV                      1
Nike Nike HPAG                   1
Name: lv_type, Length: 1417, dtype: int64

In [ ]:
launch_vehicles = pd.read_csv('D:\Bureau\Stage\BCG\PwC\Launch_vehicles_wikipedia.csv')

